In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import json
from pandas.io.json import json_normalize

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\akkrishn\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
value = input("Enter Stock Symbol:")

In [8]:
# Read your api key environment variable
from newsapi import NewsApiClient
load_dotenv()
api_key = os.getenv("NEWSAPI_KEY")

In [9]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [10]:
# Fetch the Bitcoin news articles
stock_headlines = newsapi.get_everything(q=value,
                                        language="en",
                                        page_size=100,
                                        sort_by='relevancy')
#save json
with open('stock.json', 'w') as json_file:
   json.dump(stock_headlines, json_file)

In [11]:
# Fetch the Bitcoin news articles
fed_headlines = newsapi.get_everything(q='fed',
                                        language="en",
                                        page_size=100,
                                        sort_by='relevancy')
#save json
with open('fed.json', 'w') as json_file:
   json.dump(fed_headlines, json_file)

In [12]:
# Create the Bitcoin sentiment scores DataFrame
tc = pd.read_json ('stock.json')
#parse the json and load into a dataframe
stock_df = json_normalize(stock_headlines['articles'])
#drop all columns except for description
stock_df = stock_df.drop(['author', 'title', 'url','urlToImage','publishedAt','content','source.id','source.name'], axis=1)
#instantiate vader sentiment 
sentiment_title = stock_df.apply(lambda x : analyzer.polarity_scores(x['description']), axis=1)
#add a column for each sentimement metric
stock_df['Comp'] =sentiment_title.apply(lambda x : x.get('compound') )
stock_df['Pos'] =sentiment_title.apply(lambda x : x.get('pos') )
stock_df['Neu'] =sentiment_title.apply(lambda x : x.get('neu') )
stock_df['Neg'] =sentiment_title.apply(lambda x : x.get('neg') )
stock_df

<ipython-input-12-192e310283a8>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  stock_df = json_normalize(stock_headlines['articles'])


,description,Comp,Pos,Neu,Neg
0,The biggest disappointment was its lack of gui...,-0.6808,0.000,0.663,0.337
1,It looks like investors of GameStop (NYSE: GME...,0.3612,0.088,0.912,0.000
2,GameStop (GME) doesn't possess the right combi...,0.2263,0.076,0.924,0.000
3,The biggest disappointment was its lack of gui...,0.1280,0.199,0.619,0.182
4,"To bolster e-commerce operations, GameStop (GM...",0.0000,0.000,1.000,0.000
...,...,...,...,...,...
95,"This week: Apple’s App Store is dealt a blow, ...",0.0000,0.000,1.000,0.000
96,GameStop (GME) trades with a market cap larger...,0.0000,0.000,1.000,0.000
97,It was a mixed week for meme stocks.AMC (AMC) ...,0.4588,0.100,0.900,0.000
98,"Roger Fan, chief investment officer of RF Capi...",0.3182,0.076,0.924,0.000


In [13]:
# Create the Bitcoin sentiment scores DataFrame
tc = pd.read_json ('fed.json')
#parse the json and load into a dataframe
fed_df = json_normalize(fed_headlines['articles'])
#drop all columns except for description
fed_df = fed_df.drop(['author', 'title', 'url','urlToImage','publishedAt','content','source.id','source.name'], axis=1)
#instantiate vader sentiment 
sentiment_title = fed_df.apply(lambda x : analyzer.polarity_scores(x['description']), axis=1)
#add a column for each sentimement metric
fed_df['Comp'] =sentiment_title.apply(lambda x : x.get('compound') )
fed_df['Pos'] =sentiment_title.apply(lambda x : x.get('pos') )
fed_df['Neu'] =sentiment_title.apply(lambda x : x.get('neu') )
fed_df['Neg'] =sentiment_title.apply(lambda x : x.get('neg') )
fed_df

<ipython-input-13-dee714f1f9ca>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  fed_df = json_normalize(fed_headlines['articles'])


,description,Comp,Pos,Neu,Neg
0,"From backpacks to boxer briefs, we have the co...",0.5859,0.179,0.821,0.000
1,The chainless Schaeffler Free Drive system wor...,0.8225,0.211,0.789,0.000
2,The Fed is expected to signal that it will slo...,0.4019,0.097,0.903,0.000
3,Here’s what you need to know at the end of the...,0.0000,0.000,1.000,0.000
4,Workers in China are fed up with the brutal 12...,-0.1027,0.149,0.685,0.165
...,...,...,...,...,...
95,Federal Reserve policymakers begin a two-day m...,0.4019,0.076,0.924,0.000
96,"The U.S. economy ""downshifted slightly"" in Aug...",0.0000,0.000,1.000,0.000
97,U.S. Federal Reserve bank Gov. Michelle Bowman...,-0.2263,0.064,0.874,0.062
98,Boston Federal Reserve President Eric Rosengre...,0.4019,0.063,0.937,0.000


In [14]:
# Describe the stock Sentiment
stock_df.describe()

,Comp,Pos,Neu,Neg
count,100.000000,100.000000,100.000000,100.000000
mean,0.136356,0.091800,0.866180,0.042010
std,0.377899,0.111559,0.121039,0.069904
min,-0.827100,0.000000,0.488000,0.000000
25%,-0.060325,0.000000,0.795750,0.000000
50%,0.000000,0.071000,0.884500,0.000000
75%,0.426225,0.159000,1.000000,0.076000
max,0.913600,0.504000,1.000000,0.337000


In [15]:
# Describe the fed Sentiment
fed_df.describe()

,Comp,Pos,Neu,Neg
count,100.000000,100.000000,100.000000,100.000000
mean,0.109406,0.089450,0.846500,0.064010
std,0.438896,0.085776,0.114403,0.081491
min,-0.888500,0.000000,0.530000,0.000000
25%,-0.115300,0.000000,0.781500,0.000000
50%,0.000000,0.077500,0.852500,0.000000
75%,0.458800,0.140500,0.929000,0.108500
max,0.900100,0.470000,1.000000,0.304000


In [16]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import string
from string import punctuation
import re

In [17]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '